# Neural network with Hyper-parameter fine tuning model

In [31]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.datasets import fashion_mnist
from keras_tuner import RandomSearch


In [32]:
Trails = 3  # Number of models to train
epochs = 2  # Number of epochs per model


# Get the Fashion MNIST dataset


In [33]:
# Load Fashion MNIST dataset
(x_train, y_train), (x_val, y_val) = fashion_mnist.load_data()
x_train = np.expand_dims(x_train.astype('float32') / 255., -1)
x_val = np.expand_dims(x_val.astype('float32') / 255., -1)
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)


In [34]:
def build_model_fashion_mnist(hp):
    """
    Function to build a TF model based on hyperparameter values for Fashion MNIST dataset
    Args:
        hp (hyperparameter): values

    Returns:
        Model : compiled model
    """
    num_layers = hp.Int('num_layers', 2, 8, default=6)
    lr = hp.Choice('learning_rate', [0.0001, 0.05])
    inputs = layers.Input(shape=(28, 28, 1))
    x = inputs
    for idx in range(num_layers):
        idx = str(idx)

        filters = hp.Int('filters_' + idx, 32, 256, step=32, default=64)
        x = layers.Conv2D(filters=filters, kernel_size=3, padding='same', activation='relu')(x)

        if x.shape[1] >= 8:
            pool_type = hp.Choice('pool_' + idx, values=['max', 'avg'])
            if pool_type == 'max':
                x = layers.MaxPooling2D(2)(x)
            elif pool_type == 'avg':
                x = layers.AveragePooling2D(2)(x)

    x = layers.Flatten()(x)
    outputs = layers.Dense(10, activation='softmax')(x)

    # Build model
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=Adam(lr), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [35]:

tuner_fashion_mnist = RandomSearch(build_model_fashion_mnist, objective='val_accuracy', max_trials=Trails,
                                    project_name='FashionMNIST_results')


Reloading Tuner from .\FashionMNIST_results\tuner0.json


In [36]:
tuner_fashion_mnist.search_space_summary()

Search space summary
Default search space size: 10
num_layers (Int)
{'default': 6, 'conditions': [], 'min_value': 2, 'max_value': 8, 'step': 1, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.0001, 'conditions': [], 'values': [0.0001, 0.05], 'ordered': True}
filters_0 (Int)
{'default': 64, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
pool_0 (Choice)
{'default': 'max', 'conditions': [], 'values': ['max', 'avg'], 'ordered': False}
filters_1 (Int)
{'default': 64, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
pool_1 (Choice)
{'default': 'max', 'conditions': [], 'values': ['max', 'avg'], 'ordered': False}
filters_2 (Int)
{'default': 64, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
filters_3 (Int)
{'default': 64, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
filters_4 (Int)
{'default': 64, 'conditions': [], 'min_value

In [37]:
tuner_fashion_mnist.search(x_train, y_train, batch_size=128, epochs=2, validation_data=(x_val, y_val))

Trial 3 Complete [00h 05m 06s]
val_accuracy: 0.7498999834060669

Best val_accuracy So Far: 0.7498999834060669
Total elapsed time: 00h 35m 12s


In [38]:
tuner_fashion_mnist.results_summary()

Results summary
Results in .\FashionMNIST_results
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
num_layers: 4
learning_rate: 0.05
filters_0: 96
pool_0: max
filters_1: 224
pool_1: avg
filters_2: 64
filters_3: 32
filters_4: 32
filters_5: 256
Score: 0.7498999834060669

Trial 0 summary
Hyperparameters:
num_layers: 6
learning_rate: 0.05
filters_0: 32
pool_0: max
filters_1: 256
pool_1: max
filters_2: 224
filters_3: 256
filters_4: 256
filters_5: 224
Score: 0.10000000149011612

Trial 1 summary
Hyperparameters:
num_layers: 6
learning_rate: 0.05
filters_0: 256
pool_0: max
filters_1: 128
pool_1: max
filters_2: 64
filters_3: 192
filters_4: 192
filters_5: 96
Score: 0.10000000149011612


In [41]:
best_model_fashion_mnist = tuner_fashion_mnist.get_best_models(num_models=1)[0]
best_model_fashion_mnist.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 96)        960       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 96)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 224)       193760    
                                                                 
 average_pooling2d (Average  (None, 7, 7, 224)         0         
 Pooling2D)                                                      
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          129088